**GPT-3 in-context learning**

In [1]:
import numpy as np
import pandas as pd

**Train data**

In [14]:
train_data = pd.Series(open('train_prompt.txt').readlines())
val_data = pd.Series(open('validation_prompt.txt').readlines())

**Unsupervised selection of few-shot examples**

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(train_data)

In [4]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering() #n_clusters = None, affinity = 'cosine', linkage = 'average', distance_threshold = 0.4)
clustering.fit(embeddings)

AgglomerativeClustering()

In [5]:
labels = clustering.labels_
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,

In [6]:
fewshot_indices = []
for label in np.unique(labels):
    indices = np.random.choice(train_data[labels==label].index.tolist(), size = 15)
    fewshot_indices.extend(indices)
fewshot_indices

[74,
 114,
 398,
 437,
 326,
 117,
 254,
 160,
 452,
 402,
 410,
 396,
 145,
 457,
 340,
 161,
 97,
 445,
 31,
 142,
 81,
 181,
 376,
 387,
 230,
 291,
 291,
 390,
 372,
 156]

**Few shot learning from GPT-3**

In [8]:
from prompting import predict_ns, raw_predict

In [11]:
prompt = 'Tell me what you know about Epic Kitchens dataset'

In [12]:
raw_predict(prompt)

'\n\nEpic Kitchens is a large-scale video dataset of cooking activities, annotated with action labels and object tracks. The dataset contains 100 hours of cooking videos from 30 different kitchens, captured with four synchronized cameras.'

In [69]:
val_data[0]

'take plate=>put down plate=>take pizza=>put pizza onto plate=>take pizza=>put pizza slice onto plate=>take pizza slice=>put pizza slice onto plate=>take paper=>fold paper=>take paper=>open bin=>throw paper into bin=>close bin=>take tray=>take spatula=>put down tray=>take trays=>put down trays=>put down spatula and pizza cutter=>dry hands=>take plate=>put down plate=>take tablecloth=>fold tablecloth=>put tablecloth into cupboard=>take container=>open container=>take pizza slice=>put pizza slice into container=>take pizza slice=>take knife=>put down knife=>cut pizza=>put pizza slice into container=>put down knife=>take lid=>put down lid=>close container=>turn on tap=>wash hand=>turn off tap=>take sponge=>squeeze sponge=>take washing up liquid=>put down washing up liquid=>wash knife=>take fork=>wash fork=>put down fork=>take pizza cutter=>turn on tap=>rinse pizza cutter=>wash pizza cutter=>put down pizza cutter=>take plate=>wash plate=>put down plate=>take spatula=>wash spatula=>put down

In [68]:
for i in val_data:
    print(len(i.split('=>')))

148
62
30
354
291
371
14
14
50
13
26
125
136
16
4
64
27
3
11
25
61
50
113
7
21
111
66
60
23
30
63
32
43
15
43
17
29
26
14
147
63
43
133
120
120
17
27
235
147
18
34
195
11
73
21
24
111
26
49
10
48
13
33
37
14
64
57
27
43
27
113
38
135
91
53
19
42
46
98
34
13
30
126
70
26
60
350
217
484
179
104
347
34
41
22
3
18
15
5
12
34
39
12
9
10
15
24
64
32
57
16
26
20
25
14
26
6
16
133
31
564
45
106
331
170
46
29
22
60
27
12
40
37
48
3
16
39
44


In [71]:
index = 0
separator = '=>'
splits = val_data[index].split(separator)
time_step = np.random.randint(1, 15)#len(splits)-1)
prompt = f'''The following denotes a sequence of cooking activities from the Epic Kitchens dataset. Complete the remaining sequence.

{separator.join([text for text in splits[:time_step]])}=>'''
# {'. '.join(['I ' + text for text in splits[:time_step]])}. Then I '''
print(prompt)

The following denotes a sequence of cooking activities from the Epic Kitchens dataset. Complete the remaining sequence.

take plate=>put down plate=>take pizza=>put pizza onto plate=>take pizza=>put pizza slice onto plate=>take pizza slice=>put pizza slice onto plate=>take paper=>fold paper=>take paper=>


In [72]:
time_step, splits[time_step]

(11, 'open bin')

In [73]:
raw_predict(prompt)

'fold paper=>\n\ntake paper=>fold paper=>take paper=>fold paper=>take paper=>fold paper=>take paper=>fold paper=>take paper=>fold paper=>take paper=>fold paper'

**Dataset 2:** Virtual Homes

**Zero-shot GPT-3 learning**

In [74]:
train_data = pd.Series(open('train_prompt_1.txt').readlines())
val_data = pd.Series(open('validation_prompt_1.txt').readlines())
test_data = pd.Series(open('test_prompt_1.txt').readlines())

In [75]:
train_data

0       Walk to home office=>Walk to computer=>Find co...
1       Find clothes dress=>Find towel=>Walk to bathro...
2       Find remote control=>Grab remote control=>Find...
3       Walk to home office=>Walk to clothes jacket=>F...
4       Walk to bathroom=>Walk to basket for clothes=>...
                              ...                        
3047    Walk to home office=>Walk to filing cabinet=>O...
3048    Walk to home office=>Walk to phone=>Find phone...
3049    Walk to dining room=>Walk to table=>Find table...
3050    Walk to dining room=>Walk to food food=>Find f...
3051    Walk to home office=>Walk to table=>Find table...
Length: 3052, dtype: object

In [76]:
val_data

0       Walk to bedroom=>Walk to floor lamp=>Switch of...
1       Walk to home office=>Walk to purse=>Find purse...
2       Walk to dining room=>Walk to clothes dress=>Fi...
3                                   Find cat=>Touch cat\n
4       Walk to home office=>Walk to couch=>Find couch...
                              ...                        
1011    Find television=>Turn to television=>Look at t...
1012    Walk to home office=>Walk to light=>Find light...
1013    Walk to home office=>Walk to television=>Plug ...
1014    Walk to home office=>Walk to laptop=>Find lapt...
1015    Find cup=>Turn to cup=>Point at cup=>Grab cup=...
Length: 1016, dtype: object

In [77]:
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM

In [78]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B", cache_dir = './gpt2_tok')
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B", cache_dir = './gpt_neo')

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

In [ ]:
prompt = "a,b,c,d,e,f"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

greedy_output = model.generate(input_ids = inputs["input_ids"], max_length=256)

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

**GPT-Neo finetuning**

In [ ]:
query = val_data[0]
query_instruct_text

In [ ]:
task = 

In [4]:
train_data

['open door turn on light close door open fridge take celery take container take tofu close fridge open fridge take carrots and open drawer close fridge put down vegetables open cupboard take cutting board put down cutting board close cupboard open drawer take knife take knife put down knife close drawer put down knife open tap wash courgette still washing courgette wash carrot still washing carrot close tap put down vegetables open cupboard take grater take pan put down pan close cupboard close cupboard take courgette cut courgette turn on hob cut courgette still cutting courgette dicing courgette still dicing courgette still dicing courgette still dicing courgette pour courgette onto pan throw courgette into rubbish bin open drawer close drawer take spatula stir courgette take salt open salt pour salt onto pan put down salt stir courgette put down spatula take celery still washing celery open tap wash celery close tap put down celery cut celery still cutting celery pour celery onto p